In [1]:
import numpy as np
import os
import pandas as pd
import re
import json
from algorithms import get_A


In [2]:

folder = "outputs/80"
graphs = sorted(os.listdir(folder))
results = {}
for graph in graphs:

    _,A,_ = get_A(f"k-cluster_80/{graph}.dat")
    _,_,k,_ = map(int, re.search(r'kcluster(\d+)_(\d+)_(\d+)_(\d+)', graph).groups())
    instances = sorted(os.listdir(f"{folder}/{graph}"))

    res_mu_all = {}

    for instance in instances:

        res_mu = {}
        mu_lb, mu = map(int, re.search(r'_mulb_(\d+)_mu_(\d+)', instance).groups())
        
        with open(f"{folder}/{graph}/{instance}", 'rb') as f:
            data = json.load(f)

        x = np.array(data['qubo']['x'])

        res_mu['k'] = k
        res_mu['mu (LB)'] = mu_lb
        res_mu['mu'] = mu
        res_mu['time'] = data['time']
        res_mu['computed card'] = int(data['qubo']['cardinality'])
        res_mu['BaB depth'] = res_mu['maxcut'].get('max_depth')
        res_mu['num BaB nodes'] = res_mu['maxcut'].get('eval_bab_nodes')

        with open(f"qubo_folder/80/{graph}/{instance.removesuffix(".output.json")}", 'rb') as f:
            input_data = json.load(f)
        
        res_mu['optimum'] = input_data['optimum']
        res_mu['edges'] = int(0.5*A@x@x)

        res_mu_all[mu] = res_mu
    results[graph] = res_mu_all


        


KeyError: 'maxcut'

In [4]:
df = pd.DataFrame(results['kcluster40_075_30_1'])
df

,49,51,53,55,57,59,61,63,65,67,69
k,30.00000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mu (LB),59.00000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mu,49.00000,51.000000,53.000000,55.000000,57.000000,59.000000,61.000000,63.000000,65.000000,67.000000,69.000000
time,26.78174,23.148789,28.067055,20.165518,14.647781,14.726574,12.233841,11.962094,19.412562,9.740789,10.239079
computed card,30.00000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
optimum,349.00000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000
edges,349.00000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000,349.000000


In [12]:
df_tidies = {}
for graph in graphs:
    try:
        df_raw = pd.DataFrame(results[graph])

        df_tidy = df_raw.transpose()
        df_tidy = df_tidy.sort_values(['mu'])
        df_tidy['graph'] = graph.replace("_",r"\_")

        cols = ['graph', 'k', 'optimum', 'mu (LB)', 'mu', 'time',	'computed card', 'edges']
        df_tidy = df_tidy[cols]

        int_cols = ['mu (LB)', 'mu', 'k', 'optimum', 'computed card', 'edges']
        df_tidy[int_cols] = df_tidy[int_cols].astype(int)

        df_tidy['time'] = df_tidy['time'].map(lambda x: f"{x:.3f}")

        df_tidy = df_tidy.set_index(['graph','mu (LB)','k', 'optimum', 'mu'])
        df_tidy.index.set_names(
            ['graph', r'$\mu_{LB}$', 'k', 'optimum', r'$\mu$'], inplace=True
            )
        df_tidies[graph] = df_tidy
    except:
        continue

In [13]:
df_tidies['kcluster40_025_10_1']

time  computed card  \
graph                  $\mu_{LB}$ k  optimum $\mu$                          
kcluster40\_025\_10\_1 19         10 29      9      14.420             10   
                                             11      6.780             10   
                                             13     11.501             10   
                                             15     13.773             10   
                                             17      5.434             10   
                                             19      2.993             10   
                                             21      1.825             10   
                                             23      0.643             10   
                                             25      0.250             10   
                                             27      0.253             10   
                                             29      0.187             10   

                                                    edges  
graph                  $\mu_{LB}$ k  optimum $\mu$         
kcluster40\_025\_10\_1 19         10 29      9         29  
                                             11        29  
                                             13        29  
                                             15        29  
                                             17        29  
                                             19        29  
                                             21        29  
                                             23        29  
                                             25        29  
                                             27        29  
                                             29        29

In [16]:
import os

os.makedirs("latex_table", exist_ok=True)

for graph in df_tidies.keys():
    
    df = df_tidies[graph]
    latex_tabular = df.to_latex()

    latex_full = f"""\\begin{{table}}[h]
        \\centering
        \\resizebox{{0.9\\linewidth}}{{!}}{{%
    {latex_tabular}
        }}
    \\end{{table}}"""

    # Write to file
    file_path = f"latex_table_QP_biqbin/{graph}.tex"
    with open(file_path, "w") as f:
        f.write(latex_full)

    print(f"LaTeX table written to {file_path}")


LaTeX table written to latex_table_QP_biqbin/kcluster40_025_10_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_025_20_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_050_10_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_050_20_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_050_30_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_075_10_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_075_20_1.tex
LaTeX table written to latex_table_QP_biqbin/kcluster40_075_30_1.tex


In [ ]:
for file in os.listdir("latex_table_QP_biqbin"):
    print(f"\\input{{{file}}}")

\input{kcluster40_025_20_1.tex}
\input{kcluster40_025_10_1.tex}
\input{kcluster40_050_20_1.tex}
\input{kcluster40_075_20_1.tex}
\input{kcluster40_075_10_1.tex}
\input{kcluster40_075_30_1.tex}
\input{kcluster40_050_10_1.tex}
\input{kcluster40_050_30_1.tex}
